In [3]:
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import random
import pandas as pd
import sys
from platform import python_version
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,train_test_split,cross_val_score, KFold, StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_recall_fscore_support, make_scorer
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.ensemble import AdaBoostClassifier
# ! pip install xgboost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.optimizers import Adam, SGD
! pip install "C:\Users\marcu\Desktop\GA Tech\ML\ML-SP24\mlrose"
import mlrose_hiive
from mlrose_hiive.runners import SKMLPRunner
from sklearn.preprocessing import OneHotEncoder



Processing c:\users\marcu\desktop\ga tech\ml\ml-sp24\mlrose
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mlrose-hiive: filename=mlrose_hiive-2.2.4-py3-none-any.whl size=103936 sha256=44c47c473960becf4d15471a7d801c0dce22928de48b35e2f703c8a8a3c6c5f4
  Stored in directory: C:\Users\marcu\AppData\Local\Temp\pip-ephem-wheel-cache-6zd5c2zk\wheels\1b\71\20\a26b3f48984937d91461af271cddc3ba8371491b048e63bbfa
Successfully built mlrose-hiive
  Attempting uninstall: mlrose-hiive
    Found existing installation: mlrose-hiive 2.2.4
    Uninstalling mlrose-hiive-2.2.4:
      Successfully uninstalled mlrose-hiive-2.2.4


In [2]:
def data_ingestion():
    # Ingesting Wine Quality Dataset
    wine = pd.read_csv(r'C:\Users\marcu\Desktop\GA Tech\ML\ML-SP24\Assignment 2\winequality-white.csv')
    column_names = wine.columns.tolist()
    # Standardize the values in all columns
    scaler = StandardScaler()
    columns_stand = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
    wine[columns_stand] = scaler.fit_transform(wine[columns_stand])
    
    # Encoding quality wine 
    label_encoder = LabelEncoder()
    wine['encoded_quality'] = label_encoder.fit_transform(wine['quality'])
    wine = wine.drop(columns=['quality'])
    wine = wine.rename(columns={'encoded_quality': 'quality'})


    # Descriptive Stats for Wine
    df_wine = wine.describe()
    df_wine.to_csv('descriptive/descriptive_stats_wine.csv')

    # Output boxplot for outlier distribution
    column_names = wine.columns.tolist()
    for i in column_names:
        column_data = wine[i].values
        # Create boxplot using Matplotlib
        plt.clf()
        plt.boxplot(column_data)
        plt.title(f'Box and Whisker Plot for Wine Quality for {i}')
        plt.savefig(f'image/wine_{i}.png')

    # Plotting Target Variable for Apple Quality
    plt.clf()
#     plt.hist(wine['quality'], label=i)
    plt.savefig('image/wine_target.png', bbox_inches='tight')
    
    # Ingesting Apple Quality
    aq = pd.read_csv(r'C:\Users\marcu\Desktop\GA Tech\ML\ML-SP24\Assignment 2\apple_quality.csv')
    # Descriptive Stats for Wine
    df_aq = aq.describe()
    df_aq.to_csv('descriptive/descriptive_stats_aq.csv')
    
    # Output boxplot for outlier distribution
    column_names = aq.columns.tolist()
    for i in column_names:
        column_data = aq[i].values
        # Create boxplot using Matplotlib
        plt.clf()
        plt.boxplot(column_data)
        plt.title(f'Box and Whisker Plot for Apple Quality for {i}')
        plt.savefig(f'image/aq_{i}.png')

    # Plotting Target Variable for Apple Quality
    plt.clf()
#     plt.hist(aq['Quality'], label=i)
    plt.savefig('image/apple_target.png', bbox_inches='tight')
    
    return(wine,aq)

In [3]:
def train_test(wine, aq):
    list_w = wine.columns.tolist()
    list_w.remove('quality')
    x_train_w, x_test_w, y_train_w, y_test_w = train_test_split(wine[list_w], wine['quality'], test_size=0.3, random_state=42)

    list_a = aq.columns.tolist()
    list_a.remove('Quality')
    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(aq[list_a], aq['Quality'], test_size=0.3, random_state=42)

    return(x_train_w, x_test_w, y_train_w, y_test_w, x_train_a, x_test_a, y_train_a, y_test_a)

In [9]:
def strat_kfold_ann(clf, num_folds, x_train, y_train, dataset = 'wine', epochs = 5):
    # Create a StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True,random_state = 42)
    
    # Lists to store training and validation scores for each fold
    training_scores = []
    validation_scores = []
    
    # Iterate over folds
    for fold_num, (train_index, val_index) in enumerate(stratified_kfold.split(x_train, y_train), start=1):


        # Extract training and validation sets for this fold
        X_train_sub, X_val_sub = x_train.iloc[train_index], x_train.iloc[val_index]
        y_train_sub, y_val_sub = y_train.iloc[train_index], y_train.iloc[val_index]
        
        
        # One hot encoding
        encoder = OneHotEncoder(sparse=False)
        y_train_hot = encoder.fit_transform(y_train_sub.values.reshape(-1, 1))
        y_val_hot = encoder.transform(y_val_sub.values.reshape(-1, 1))
        
        
        # Train the Decision Tree model
        clf.fit(X_train_sub, y_train_hot)

        if dataset == 'wine':
            # Make predictions on the training set
            predictions_train = clf.predict(X_train_sub)
            y_train_pred = np.argmax(predictions_train, axis=1)
            
            # Converting y_train_sub into the quality category 
            y_train_sub = np.argmax(y_train_hot, axis=1)
            
            # Make predictions on the validation set
            predictions_val = clf.predict(X_val_sub)
            y_val_pred = np.argmax(predictions_val, axis=1)
            
            # Converting y_train_sub into the quality category 
            y_val_sub = np.argmax(y_val_hot, axis=1)

        else:            
            # Make predictions on the training set
            predictions_train = clf.predict(X_train_sub)
            y_train_pred = np.argmax(predictions_train, axis=1)
            
            # Converting y_train_sub into the quality category 
            y_train_sub = np.argmax(y_train_hot, axis=1)
            
            # Make predictions on the validation set
            predictions_val = clf.predict(X_val_sub)
            y_val_pred = np.argmax(predictions_val, axis=1)
            
            # Converting y_train_sub into the quality category 
            y_val_sub = np.argmax(y_val_hot, axis=1)

        # Calculate recall scores for training set
        _, recall_train, _, _ = precision_recall_fscore_support(y_train_sub, y_train_pred, average='micro')
        training_scores.append(recall_train)

        # Calculate recall scores for validation sets
        _, recall_val, _, _ = precision_recall_fscore_support(y_val_sub, y_val_pred, average='micro')
        validation_scores.append(recall_val)

    # Print mean scores across all folds
    mean_training_score = np.mean(training_scores)
    mean_validation_score = np.mean(validation_scores)
#     print(mean_validation_score, mean_training_score)
    return(mean_validation_score, mean_training_score)

tf.random.set_seed(42)

In [10]:
# Data ingestion
wine,aq = data_ingestion()
x_train_w, x_test_w, y_train_w, y_test_w, x_train_a, x_test_a, y_train_a, y_test_a = train_test(wine,aq)

<Figure size 640x480 with 0 Axes>

# Optimal Hyperparameters for GA

In [14]:
pop_size = [700,500,300,100]
mutation_prob = [0.7,0.5,0.1,0.3]

for i in range(0,4):
    for x in range(0,4):
        start_time = time.time()
        nn_model_base = mlrose_hiive.NeuralNetwork([64,64,64], activation = 'tanh', 
                         algorithm = 'genetic_alg', max_iters = 1000, 
                         bias = True, is_classifier = True, learning_rate = 0.0001, 
                         early_stopping = True, clip_max = 5, max_attempts = 100, 
                         random_state = 42, curve = True, pop_size = pop_size[i], mutation_prob = mutation_prob[x])
        val, train = strat_kfold_ann(nn_model_base, num_folds=2, x_train = x_train_a, y_train = y_train_a, dataset = 'apple', epochs = 5)
        end_time = time.time()
        wall_clock_time = end_time - start_time
        print('pop_size: ', pop_size[i], 'mutation_prob: ', mutation_prob[x] , 'validation recall: ', val , 'training recall: ', train,  'wall clock time: ', wall_clock_time)


pop_size:  700 mutation_prob:  0.7 validation recall:  0.6142857142857143 training recall:  0.6214285714285714 wall clock time:  1487.3166959285736
pop_size:  700 mutation_prob:  0.5 validation recall:  0.5985714285714285 training recall:  0.6028571428571429 wall clock time:  7169.323969841003
pop_size:  700 mutation_prob:  0.1 validation recall:  0.5892857142857143 training recall:  0.6239285714285714 wall clock time:  5033.533531188965
pop_size:  700 mutation_prob:  0.3 validation recall:  0.5932142857142857 training recall:  0.5903571428571428 wall clock time:  10420.80622124672
pop_size:  500 mutation_prob:  0.7 validation recall:  0.5925 training recall:  0.6253571428571428 wall clock time:  7773.548013687134
pop_size:  500 mutation_prob:  0.5 validation recall:  0.6467857142857143 training recall:  0.6375 wall clock time:  10689.922334909439
pop_size:  500 mutation_prob:  0.1 validation recall:  0.5742857142857143 training recall:  0.6042857142857143 wall clock time:  14314.44396

In [4]:


# Define the fitness function for the Flip Flop problem
fitness_flip_flop = mlrose_hiive.FlipFlop()

# Define the optimization problem
problem = mlrose_hiive.DiscreteOpt(length=100, fitness_fn=fitness_flip_flop, maximize=True, max_val=2)

# Define the initial state
init_state = np.random.randint(2, size=100)

# Define the decay schedule for simulated annealing
schedule = mlrose_hiive.GeomDecay(init_temp = 1e10, decay = 0.35)

# Run simulated annealing to optimize the problem
best_state, best_fitness, a = mlrose_hiive.simulated_annealing(problem, schedule=schedule, max_attempts=10, max_iters=1000, init_state=init_state, random_state=42)

print("Best State:", best_state)
print("Best Fitness:", best_fitness)

Best State: [0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0
 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1
 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1]
Best Fitness: 79.0


In [5]:
# Run simulated annealing to optimize the problem
best_state, best_fitness, fitness_curve = mlrose_hiive.random_hill_climb(problem, 
                                                                          max_attempts = 100,
                                                                          max_iters = 1000,
                                                                          restarts = 0,
                                                                          init_state = init_state,
                                                                          curve = True,
                                                                          random_state = 42)
print(best_state, best_fitness)
best_state, best_fitness, a = mlrose_hiive.simulated_annealing(problem, schedule=schedule, max_attempts=10, max_iters=1000, curve= True, init_state=init_state, random_state=42)

print(best_state, best_fitness)

best_state, best_fitness, fitness_curve = mlrose_hiive.genetic_alg(problem, 

                                                                   pop_size = 100,
                                                                   mutation_prob = 0.5, 
                                                                   max_attempts = 100,
                                                                   max_iters = 1000,
                                                                   curve = True,
                                                                   random_state = 42)
print(best_state, best_fitness)

best_state, best_fitness, a = mlrose_hiive.mimic(problem, 
                                                 pop_size = 100, 
                                                 keep_pct = 0.9,
                                                 max_attempts = 100,
                                                 max_iters = 1000,
                                                 curve = True,
                                                 random_state = 42)
print(best_state, best_fitness)


[1 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1] 74.0
[0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0
 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1
 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1] 79.0
[0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1] 91.0
[0 1 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 1 1 1
 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1
 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 0] 72.0


In [6]:
runner = mlrose_hiive.MIMICRunner(problem=problem,
                                experiment_name=f'aa_mimic.csv',
                                #                           output_directory='.',
                                seed=42,
                                iteration_list=[100,200,300,400,500],
                                max_attempts=100,
                                keep_percent_list=[0.1,0.2,0.3,0.4,0.5],
                                population_sizes = [100,200,300,400,500])

df_run_stats, df_run_curves = runner.run()
df_run_stats.to_csv('aa_mimic.csv')
